## Activity 2

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Initially, we will examine performance for both classifiers using the default parameters and utilizing different scoring measures.

In [2]:
wine = load_wine()
iris = load_iris()
scoring = ['accuracy','precision_macro','recall_macro','f1_macro']

def cross_val(classifier,dataset):
    dataset.data = StandardScaler().fit_transform(dataset.data)
    return cross_validate(classifier,dataset.data,dataset.target,cv=10,scoring=scoring)



In [5]:
pd.DataFrame.from_dict(cross_val(KNeighborsClassifier(),wine),orient='columns')

,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
0,0.000000,0.004008,0.944444,0.958333,0.933333,0.940741
1,0.000000,0.003995,0.944444,0.952381,0.952381,0.948718
2,0.000000,0.003998,0.944444,0.952381,0.952381,0.948718
3,0.003992,0.004047,0.944444,0.944444,0.952381,0.944056
4,0.000000,0.007972,1.000000,1.000000,1.000000,1.000000
5,0.000000,0.008001,0.944444,0.952381,0.952381,0.948718
6,0.000000,0.007991,1.000000,1.000000,1.000000,1.000000
7,0.003947,0.004047,1.000000,1.000000,1.000000,1.000000
8,0.003988,0.003995,0.941176,0.933333,0.952381,0.937322
9,0.003995,0.003995,0.941176,0.944444,0.958333,0.947475


In [26]:
cross_val(SVC(),wine)
pd.DataFrame.from_dict(cross_val(SVC(),wine),orient='columns')

,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
0,0.003994,0.003996,0.944444,0.958333,0.933333,0.940741
1,0.000000,0.004031,1.000000,1.000000,1.000000,1.000000
2,0.003994,0.000000,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.003996,0.944444,0.944444,0.952381,0.944056
4,0.003994,0.003997,0.944444,0.958333,0.944444,0.947475
5,0.003993,0.004008,0.944444,0.958333,0.933333,0.940741
6,0.003983,0.004006,1.000000,1.000000,1.000000,1.000000
7,0.003999,0.007985,1.000000,1.000000,1.000000,1.000000
8,0.003996,0.003994,1.000000,1.000000,1.000000,1.000000
9,0.003996,0.008003,1.000000,1.000000,1.000000,1.000000


In [27]:
pd.DataFrame.from_dict(cross_val(KNeighborsClassifier(),iris),orient='columns')

,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
0,0.000000,0.003995,1.000000,1.000000,1.000000,1.000000
1,0.000000,0.008014,0.933333,0.944444,0.933333,0.932660
2,0.000000,0.004003,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.003994,0.933333,0.944444,0.933333,0.932660
4,0.000000,0.003997,0.866667,0.904762,0.866667,0.861111
5,0.003994,0.000000,0.933333,0.944444,0.933333,0.932660
6,0.000000,0.003980,0.866667,0.866667,0.866667,0.866667
7,0.000000,0.003997,1.000000,1.000000,1.000000,1.000000
8,0.000000,0.003990,1.000000,1.000000,1.000000,1.000000
9,0.000000,0.003994,1.000000,1.000000,1.000000,1.000000


In [28]:
pd.DataFrame.from_dict(cross_val(SVC(),iris),orient='columns')

,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
0,0.000000,0.003993,1.000000,1.000000,1.000000,1.000000
1,0.000000,0.003996,0.933333,0.944444,0.933333,0.932660
2,0.000000,0.004028,1.000000,1.000000,1.000000,1.000000
3,0.003960,0.000000,0.933333,0.944444,0.933333,0.932660
4,0.000000,0.004035,1.000000,1.000000,1.000000,1.000000
5,0.003954,0.004045,0.933333,0.944444,0.933333,0.932660
6,0.000000,0.003992,0.866667,0.866667,0.866667,0.866667
7,0.000000,0.004027,1.000000,1.000000,1.000000,1.000000
8,0.003988,0.003971,1.000000,1.000000,1.000000,1.000000
9,0.000000,0.004039,1.000000,1.000000,1.000000,1.000000


Next, we'll use grid search in order to fine-tune the classifiers' parameters.

In [29]:
class Grid:    
    
    def __init__(self,dataset,classifier):
        self.dataset=dataset
        self.classifier=classifier
        self.scaler= StandardScaler()
        self.scoring = {'accuracy':'accuracy','precision':'precision_macro','recall':'recall_macro','f1':'f1_macro'}
     
    def tuning(self):
        if isinstance(self.classifier, KNeighborsClassifier):
            param_grid = {'n_neighbors': [3, 5, 9],  
                          'weights': ['uniform', 'distance'],  
                          'p': [1, 2]}
                    
            grid = GridSearchCV(self.classifier,
                                param_grid,
                                cv=10,
                                refit='accuracy',
                                scoring=self.scoring)

            grid.fit(self.scaler.fit_transform(self.dataset.data),self.dataset.target)
            print(f'Best parameters found are: {grid.best_params_}, with a score of {grid.best_score_}')      
            return pd.DataFrame.from_dict(grid.cv_results_,orient='columns')

        if isinstance(self.classifier, SVC):
            param_grid = {'C': [0.1, 1, 10],
                          'kernel': ['linear', 'rbf', 'poly']}

            grid = GridSearchCV(self.classifier,
                                param_grid,
                                cv=10,
                                refit='accuracy',
                                scoring=self.scoring)

            grid.fit(self.scaler.fit_transform(self.dataset.data),self.dataset.target)
            print(f'Best parameters found are: {grid.best_params_}, with a score of {grid.best_score_}')
            return pd.DataFrame.from_dict(grid.cv_results_,orient='columns')

In [30]:
wine_knn = Grid(load_wine(),KNeighborsClassifier())
wine_knn.tuning()

Best parameters found are: {'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}, with a score of 0.9777777777777779


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,split5_test_accuracy,split6_test_accuracy,split7_test_accuracy,split8_test_accuracy,split9_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,split5_test_f1,split6_test_f1,split7_test_f1,split8_test_f1,split9_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.000799,0.001597,0.003995,8.609519e-07,3,1,uniform,"{'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}",0.888889,0.944444,0.944444,1.000000,1.0,1.000000,1.0,1.0,0.882353,0.941176,0.960131,0.044636,9,0.896825,0.952381,0.952381,1.000000,1.0,1.000000,1.0,1.0,0.885714,0.944444,0.963175,0.042151,9,0.904762,0.952381,0.952381,1.000000,1.0,1.000000,1.0,1.0,0.904762,0.958333,0.967262,0.037005,8,0.888500,0.948718,0.948718,1.000000,1.0,1.000000,1.0,1.0,0.881766,0.947475,0.961518,0.044296,9
1,0.000802,0.001604,0.003192,1.596271e-03,3,1,distance,"{'n_neighbors': 3, 'p': 1, 'weights': 'distance'}",0.888889,0.944444,0.944444,1.000000,1.0,1.000000,1.0,1.0,0.882353,0.941176,0.960131,0.044636,9,0.896825,0.952381,0.952381,1.000000,1.0,1.000000,1.0,1.0,0.885714,0.944444,0.963175,0.042151,9,0.904762,0.952381,0.952381,1.000000,1.0,1.000000,1.0,1.0,0.904762,0.958333,0.967262,0.037005,8,0.888500,0.948718,0.948718,1.000000,1.0,1.000000,1.0,1.0,0.881766,0.947475,0.961518,0.044296,9
2,0.001200,0.001833,0.004391,1.199342e-03,3,2,uniform,"{'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}",0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.0,0.882353,0.941176,0.954575,0.034784,11,0.952381,0.952381,0.952381,0.944444,1.0,0.944444,1.0,1.0,0.885714,0.944444,0.957619,0.033409,11,0.952381,0.952381,0.952381,0.952381,1.0,0.952381,1.0,1.0,0.904762,0.958333,0.962500,0.028428,11,0.948718,0.948718,0.948718,0.944056,1.0,0.944056,1.0,1.0,0.881766,0.947475,0.956351,0.034401,11
3,0.001199,0.001831,0.004396,1.198581e-03,3,2,distance,"{'n_neighbors': 3, 'p': 2, 'weights': 'distance'}",0.944444,0.944444,0.944444,0.944444,1.0,0.944444,1.0,1.0,0.882353,0.941176,0.954575,0.034784,11,0.952381,0.952381,0.952381,0.944444,1.0,0.944444,1.0,1.0,0.885714,0.944444,0.957619,0.033409,11,0.952381,0.952381,0.952381,0.952381,1.0,0.952381,1.0,1.0,0.904762,0.958333,0.962500,0.028428,11,0.948718,0.948718,0.948718,0.944056,1.0,0.944056,1.0,1.0,0.881766,0.947475,0.956351,0.034401,11
4,0.001198,0.001830,0.009190,1.831791e-03,5,1,uniform,"{'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}",0.944444,0.944444,0.944444,0.944444,1.0,1.000000,1.0,1.0,0.941176,1.000000,0.971895,0.028120,3,0.952381,0.952381,0.952381,0.944444,1.0,1.000000,1.0,1.0,0.933333,1.000000,0.973492,0.027031,3,0.952381,0.952381,0.952381,0.952381,1.0,1.000000,1.0,1.0,0.952381,1.000000,0.976190,0.023810,3,0.948718,0.948718,0.948718,0.944056,1.0,1.000000,1.0,1.0,0.937322,1.000000,0.972753,0.027430,3
5,0.001997,0.001997,0.005592,3.661228e-03,5,1,distance,"{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}",0.944444,0.944444,0.944444,0.944444,1.0,1.000000,1.0,1.0,0.941176,1.000000,0.971895,0.028120,3,0.952381,0.952381,0.952381,0.944444,1.0,1.000000,1.0,1.0,0.933333,1.000000,0.973492,0.027031,3,0.952381,0.952381,0.952381,0.952381,1.0,1.000000,1.0,1.0,0.952381,1.000000,0.976190,0.023810,3,0.948718,0.948718,0.948718,0.944056,1.0,1.000000,1.0,1.0,0.937322,1.000000,0.972753,0.027430

In [31]:
wine_svc = Grid(load_wine(),SVC())
wine_svc.tuning()
#The warning does not seem to affect the overall measure

C:\Users\gsevr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters found are: {'C': 1, 'kernel': 'rbf'}, with a score of 0.9777777777777779


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,split5_test_accuracy,split6_test_accuracy,split7_test_accuracy,split8_test_accuracy,split9_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,split5_test_f1,split6_test_f1,split7_test_f1,split8_test_f1,split9_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.003995,3.162980e-07,0.000000,0.000000,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.944444,1.000000,0.944444,0.944444,1.000000,1.000000,1.000000,1.000000,0.941176,1.000000,0.977451,0.027631,3,0.958333,1.000000,0.952381,0.944444,1.000000,1.000000,1.000000,1.000000,0.933333,1.000000,0.978849,0.026573,3,0.933333,1.000000,0.952381,0.952381,1.000000,1.000000,1.000000,1.000000,0.952381,1.000000,0.979048,0.026186,1,0.940741,1.000000,0.948718,0.944056,1.000000,1.000000,1.000000,1.000000,0.937322,1.000000,0.977084,0.028193,2
1,0.003197,1.598589e-03,0.002798,0.001832,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.944444,1.000000,1.000000,0.944444,0.944444,0.944444,1.000000,0.944444,1.000000,1.000000,0.972222,0.027778,4,0.958333,1.000000,1.000000,0.944444,0.958333,0.958333,1.000000,0.958333,1.000000,1.000000,0.977778,0.022567,4,0.933333,1.000000,1.000000,0.952381,0.944444,0.933333,1.000000,0.944444,1.000000,1.000000,0.970794,0.029663,4,0.940741,1.000000,1.000000,0.944056,0.947475,0.940741,1.000000,0.947475,1.000000,1.000000,0.972049,0.028032,4
2,0.001596,1.954672e-03,0.002799,0.001832,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.666667,0.888889,0.722222,0.722222,0.611111,0.833333,0.555556,0.833333,0.941176,0.941176,0.771569,0.129310,9,0.846154,0.925926,0.861111,0.861111,0.500000,0.900000,0.822222,0.900000,0.958333,0.962963,0.853782,0.125957,9,0.622222,0.877778,0.677778,0.711111,0.600000,0.833333,0.522222,0.833333,0.944444,0.933333,0.755556,0.140984,9,0.611111,0.890993,0.659755,0.708577,0.518519,0.830065,0.497835,0.830065,0.947475,0.943355,0.743775,0.159692,9
3,0.000400,1.199484e-03,0.003595,0.001198,1,linear,"{'C': 1, 'kernel': 'linear'}",0.888889,0.944444,0.888889,0.944444,1.000000,0.888889,1.000000,1.000000,0.882353,1.000000,0.943791,0.050454,8,0.925926,0.944444,0.904762,0.944444,1.000000,0.896825,1.000000,1.000000,0.885714,1.000000,0.950212,0.044326,8,0.866667,0.952381,0.885714,0.952381,1.000000,0.904762,1.000000,1.000000,0.904762,1.000000,0.946667,0.050070,8,0.875000,0.944056,0.889703,0.944056,1.000000,0.888500,1.000000,1.000000,0.881766,1.000000,0.942308,0.052132,8
4,0.002395,1.955601e-03,0.002798,0.001831,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.944444,1.000000,1.000000,0.944444,0.944444,0.944444,1.000000,1.000000,1.000000,1.000000,0.977778,0.027217,1,0.958333,1.000000,1.000000,0.944444,0.958333,0.958333,1.000000,1.000000,1.000000,1.000000,0.981944,0.022438,1,0.933333,1.000000,1.000000,0.952381,0.944444,0.933333,1.000000,1.000000,1.000000,1.000000,0.976349,0.029410,3,0.940741,1.000000,1.000000,0.944056,0.947475,0.940741,1.000000,1.000000,1.000000,1.000000,0.977301,0.027856,1
5,0.003995,7.151922e-06,0.007186,0.001596,1,poly,"{'C': 1, 'kernel': 'poly'}",0.888889,1.000000,0.944444,0.944444,1.000000,1.000000,0.833333,0.944444,0.941176,1.000000,0.949673,0.052455,6,0.925926,1.000000,0.952381,0.958333,1.000000,1.000000,0.900000,0.958333,0.952381,1.000000,0.964735,0.033254,5,0.866667,1.000000,0.952381,0.944444,

In [32]:
iris_knn = Grid(load_iris(),KNeighborsClassifier())
iris_knn.tuning()

Best parameters found are: {'n_neighbors': 9, 'p': 2, 'weights': 'distance'}, with a score of 0.9666666666666666


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,split5_test_accuracy,split6_test_accuracy,split7_test_accuracy,split8_test_accuracy,split9_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,split5_test_f1,split6_test_f1,split7_test_f1,split8_test_f1,split9_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.001599,0.001958,0.009986,0.002005,3,1,uniform,"{'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}",1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.866667,1.000000,1.0,1.000000,0.953333,0.052068,2,1.0,0.944444,1.0,0.944444,0.904762,0.944444,0.866667,1.000000,1.0,1.000000,0.960476,0.045261,2,1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.866667,1.000000,1.0,1.000000,0.953333,0.052068,2,1.0,0.93266,1.0,0.93266,0.861111,0.93266,0.866667,1.00000,1.0,1.00000,0.952576,0.053086,3
1,0.000795,0.001591,0.003598,0.001200,3,1,distance,"{'n_neighbors': 3, 'p': 1, 'weights': 'distance'}",1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.866667,1.000000,1.0,1.000000,0.953333,0.052068,2,1.0,0.944444,1.0,0.944444,0.904762,0.944444,0.866667,1.000000,1.0,1.000000,0.960476,0.045261,2,1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.866667,1.000000,1.0,1.000000,0.953333,0.052068,2,1.0,0.93266,1.0,0.93266,0.861111,0.93266,0.866667,1.00000,1.0,1.00000,0.952576,0.053086,3
2,0.000802,0.001604,0.005183,0.001837,3,2,uniform,"{'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}",1.0,0.933333,1.0,0.933333,0.866667,1.000000,0.800000,1.000000,1.0,1.000000,0.953333,0.066999,2,1.0,0.944444,1.0,0.944444,0.904762,1.000000,0.805556,1.000000,1.0,1.000000,0.959921,0.060818,6,1.0,0.933333,1.0,0.933333,0.866667,1.000000,0.800000,1.000000,1.0,1.000000,0.953333,0.066999,2,1.0,0.93266,1.0,0.93266,0.861111,1.00000,0.797980,1.00000,1.0,1.00000,0.952441,0.068230,7
3,0.000799,0.001598,0.003199,0.001600,3,2,distance,"{'n_neighbors': 3, 'p': 2, 'weights': 'distance'}",1.0,0.933333,1.0,0.933333,0.866667,1.000000,0.800000,1.000000,1.0,1.000000,0.953333,0.066999,2,1.0,0.944444,1.0,0.944444,0.904762,1.000000,0.805556,1.000000,1.0,1.000000,0.959921,0.060818,6,1.0,0.933333,1.0,0.933333,0.866667,1.000000,0.800000,1.000000,1.0,1.000000,0.953333,0.066999,2,1.0,0.93266,1.0,0.93266,0.861111,1.00000,0.797980,1.00000,1.0,1.00000,0.952441,0.068230,7
4,0.000398,0.001195,0.004392,0.001213,5,1,uniform,"{'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}",1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.800000,1.000000,1.0,1.000000,0.946667,0.065320,9,1.0,0.944444,1.0,0.944444,0.904762,0.944444,0.805556,1.000000,1.0,1.000000,0.954365,0.059425,9,1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.800000,1.000000,1.0,1.000000,0.946667,0.065320,9,1.0,0.93266,1.0,0.93266,0.861111,0.93266,0.797980,1.00000,1.0,1.00000,0.945707,0.066505,9
5,0.002001,0.002001,0.007578,0.003314,5,1,distance,"{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}",1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.800000,1.000000,1.0,1.000000,0.946667,0.065320,9,1.0,0.944444,1.0,0.944444,0.904762,0.944444,0.805556,1.000000,1.0,1.000000,0.954365,0.059425,9,1.0,0.933333,1.0,0.933333,0.866667,0.933333,0.800000,1.000000,1.0,1.000000,0.946667,0.065320,9,1.0,0.93266,1.0,0.93266,0.861111,0.93266,0.797980,1.00000,1.0,1.00000,0.945707,0.066505,9
6,0.002404,0.001963,0.011584,0.001198,5,2,uniform,"{'n_neig

In [33]:
iris_svc = Grid(load_iris(),SVC())
iris_svc.tuning()

Best parameters found are: {'C': 10, 'kernel': 'poly'}, with a score of 0.9733333333333334


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,split5_test_accuracy,split6_test_accuracy,split7_test_accuracy,split8_test_accuracy,split9_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,split5_test_f1,split6_test_f1,split7_test_f1,split8_test_f1,split9_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.002398,0.001958,0.006783,0.001827,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",1.000000,0.933333,1.000000,1.000000,1.000000,0.933333,0.866667,0.933333,1.000000,1.000000,0.966667,0.044721,2,1.000000,0.944444,1.000000,1.000000,1.000000,0.944444,0.866667,0.944444,1.000000,1.000000,0.970000,0.042470,3,1.000000,0.933333,1.000000,1.000000,1.000000,0.933333,0.866667,0.933333,1.000000,1.000000,0.966667,0.044721,2,1.000000,0.93266,1.00000,1.000000,1.000000,0.932660,0.866667,0.932660,1.00000,1.00000,0.966465,0.044873,2
1,0.001198,0.001830,0.003595,0.002150,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.866667,0.933333,0.933333,0.933333,0.933333,0.933333,0.866667,0.866667,1.000000,1.000000,0.926667,0.046667,8,0.904762,0.944444,0.944444,0.944444,0.944444,0.944444,0.904762,0.866667,1.000000,1.000000,0.939841,0.038936,8,0.866667,0.933333,0.933333,0.933333,0.933333,0.933333,0.866667,0.866667,1.000000,1.000000,0.926667,0.046667,8,0.861111,0.93266,0.93266,0.932660,0.932660,0.932660,0.861111,0.866667,1.00000,1.00000,0.925219,0.048072,8
2,0.000799,0.001599,0.003198,0.001599,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.866667,0.933333,0.733333,0.866667,0.866667,0.866667,0.866667,0.866667,0.933333,0.733333,0.853333,0.065320,9,0.904762,0.944444,0.851852,0.904762,0.904762,0.904762,0.904762,0.904762,0.944444,0.851852,0.902116,0.029459,9,0.866667,0.933333,0.733333,0.866667,0.866667,0.866667,0.866667,0.866667,0.933333,0.733333,0.853333,0.065320,9,0.861111,0.93266,0.68254,0.861111,0.861111,0.861111,0.861111,0.861111,0.93266,0.68254,0.839707,0.083326,9
3,0.001602,0.001962,0.004786,0.001586,1,linear,"{'C': 1, 'kernel': 'linear'}",1.000000,0.933333,1.000000,1.000000,0.866667,1.000000,0.866667,1.000000,1.000000,1.000000,0.966667,0.053748,2,1.000000,0.944444,1.000000,1.000000,0.904762,1.000000,0.866667,1.000000,1.000000,1.000000,0.971587,0.046756,2,1.000000,0.933333,1.000000,1.000000,0.866667,1.000000,0.866667,1.000000,1.000000,1.000000,0.966667,0.053748,2,1.000000,0.93266,1.00000,1.000000,0.861111,1.000000,0.866667,1.000000,1.00000,1.00000,0.966044,0.054838,4
4,0.001601,0.001961,0.005190,0.001835,1,rbf,"{'C': 1, 'kernel': 'rbf'}",1.000000,0.933333,1.000000,0.933333,1.000000,0.933333,0.866667,1.000000,1.000000,1.000000,0.966667,0.044721,4,1.000000,0.944444,1.000000,0.944444,1.000000,0.944444,0.866667,1.000000,1.000000,1.000000,0.970000,0.042470,3,1.000000,0.933333,1.000000,0.933333,1.000000,0.933333,0.866667,1.000000,1.000000,1.000000,0.966667,0.044721,2,1.000000,0.93266,1.00000,0.932660,1.000000,0.932660,0.866667,1.000000,1.00000,1.00000,0.966465,0.044873,2
5,0.001994,0.001994,0.003203,0.001601,1,poly,"{'C': 1, 'kernel': 'poly'}",1.000000,0.933333,1.000000,1.000000,0.933333,0.866667,0.866667,0.866667,1.000000,0.933333,0.940000,0.055377,7,1.000000,0.944444,1.000000,1.000000,0.944444,0.904762,0.904762,0.904762,1.000000,0.944444,0.954762,0.040007,7,1.000000,0.933333,1.000000,1.000000,0.933333,0.866667,0.866667,0.866667,1.000000

Based on the above we can see both algorithms performing comparably on both dataset. It is important to note that iris is a dataset with few attributes. Therefore, it would be prudent to use kNN for the iris set, and SVM for wine, which is a set with significantly more features. Such sets, can affect the performance of the kNN algorithm. That was not observed here, and might  be due to the standardization of the values.